This ipynb shows an example of using `get_1site` API to solve the "1-site-N-wells" problem.  
In this example, there are 4 wells to be drilled from 1 site.  
The optimization goal is to find the best drill site location with 4 optimized wellbore trajectories,   
so that the total cost(length) of the 4 wellbores is minimum.

API url:  
url = "https://home-test.make234.com/api/v1/get_1site"

# 0. Initiate environment

In [1]:
import requests
import json
import numpy as np
from plotly import graph_objects as go
import time

In [2]:
import sys
import os
rootpath=os.getcwd().split("WelLayout_API") [0] + "\WelLayout_API"
print(rootpath)
sys.path.append(rootpath)

from tools.input2json import input2json
from tools.PlotSurvey_plotly import PlotSurvey_plotly as PlotSurvey
from tools.PlotContour_plotly import PlotContour_plotly as PlotContour

d:\OneDrive\OneDrive - LHG\Projects\\WelLayout_API


# 1. Prepare Data

## 1.1 Target information
The completion intervals (targets) are normally decided by the reservoir engineers based on the reservoir layer's trend to maximize the total production of the wells.  
Here we need the location of the first point of the completion interval($PT$) and its desired entry direction($VT$) for each well.

Following are the data for 4 targets. Multiple wells' information is gathered in a Matrix.

In [3]:
# 4 wells to be designed
n=4

# location of the first point of the completion interval (PT), [X, Y, Z]
PTM= np.array ([[517167.9, 6782513.9, -1550.0], 
                [518255.3, 6784864.0, -1559.0],
                [516179.0, 6782799.0, -1557.0],
                [515726.0, 6784079.0, -1577.0]])

# drilling direction at the first point of the completion interval (VT),
VTM= np.array ([[968.2, 636.1, -14.0],
                [-851.30, 692.0, -1.0],
                [258.0, 1055.0, -23.0],
                [541.0, 1051.0, -4.0]])


## 1.2 KOP information
The KOP depth is normally determined by the drilling engineer based on the rock mechanical property of the formation to ensure the wellbore stabilty.  
Normally, the direction at KOP is vertically downwards.

In [4]:
# location of the KOP (PK), [X, Y, Z]
# set X, Y as np.nan, so that the API can find the best site location
# KOP is beneath site
PKM= np.array ([[np.nan, np.nan, -400.0],
                [np.nan, np.nan, -400.0],
                [np.nan, np.nan, -400.0],
                [np.nan, np.nan, -400.0]])

# drilling direction at the KOP (VK), [X, Y, Z]
VKM= np.array ([[0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0]])

## 1.3 DLS constraint
Dogleg severity (DLS) is constrained by the drilling equipment.  
Drilling engineers need to set a maximum allowable DLS to ensure the wellbore stability.
DLS unit is given in `°/30m`

In [5]:
# DLS at the buildup section after the KOP 
dls_KOP=2.0

# DLS at the landing section before the Target
dls_Target=3.5

# format DLS for API
DLSM= np.array ([[dls_KOP, dls_Target],
                [dls_KOP, dls_Target],
                [dls_KOP, dls_Target],
                [dls_KOP, dls_Target]])

# optional: get the corresponding curvature radius of the DLS
rM= 30*180/DLSM/np.pi
print(rM)

[[859.4366927  491.10668154]
 [859.4366927  491.10668154]
 [859.4366927  491.10668154]
 [859.4366927  491.10668154]]


## 1.4 Compile data for API

In [6]:
# save to current path, file name "input.json"
filepath="input.json"

In [7]:
input2json(n,                
        PTM,
        VTM,
        PKM,
        VKM,
        DLSM,
        filepath=filepath)

=====file input.json written successfully=====


'{"FIELDOPT INPUT BLOCK": {"n": {"DESCRIPTION": "number of wells", "UNIT": "", "VALUE": 4}, "WellNo": {"DESCRIPTION": "well index", "UNIT": "", "VALUE": null}, "tag": {"DESCRIPTION": "well tag", "UNIT": "", "VALUE": null}, "PTM": {"DESCRIPTION": "target location, i.e., the 1st point of completion interval. 3D, [EAST,NORTH,Depth]", "UNIT": "m", "VALUE": [[517167.9, 6782513.9, -1550.0], [518255.3, 6784864.0, -1559.0], [516179.0, 6782799.0, -1557.0], [515726.0, 6784079.0, -1577.0]]}, "VTM": {"DESCRIPTION": "driling direction at the target, 3D, [EAST,NORTH,Depth]", "UNIT": "m", "VALUE": [[968.2, 636.1, -14.0], [-851.3, 692.0, -1.0], [258.0, 1055.0, -23.0], [541.0, 1051.0, -4.0]]}, "PKM": {"DESCRIPTION": "kickoff point, [East, North, Depth]", "UNIT": "m", "VALUE": [[null, null, -400.0], [null, null, -400.0], [null, null, -400.0], [null, null, -400.0]]}, "VKM": {"DESCRIPTION": "driling direction at the KOP, 3D, [EAST,NORTH,Depth]", "UNIT": "m", "VALUE": [[0.0, 0.0, -1.0], [0.0, 0.0, -1.0], [

# 2. Use the API to do the computation

In [8]:
with open(filepath) as json_file:
    input_data = json.load(json_file)

# Set request headers
headers = {
    'Content-Type': 'application/json'
}


In [ ]:
# API url
url ="https://home-test.make234.com/api/v1/get_1site"

tic=time.time()
print(f"input_data: {input_data}")
try:
    response = requests.post(url, json=input_data, headers=headers, timeout=300)
    
    # Check response status
    response.raise_for_status()
    
    # Print response results
    print("Status code:", response.status_code)
    print("Response content:")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

    # Save response content to JSON file
    with open('output.json', 'w', encoding='utf-8') as f:
        json.dump(response.json(), f, indent=2, ensure_ascii=False)
    print("Response content has been saved to output.json")
    print(url)
    
except requests.exceptions.Timeout:
    print("Request timeout (exceeded 5 minutes)")
except requests.exceptions.RequestException as e:
    print("Request error:", e)
except json.JSONDecodeError:
    print("Response is not valid JSON format")
    print("Raw response:", response.text)
toc=time.time()
print(f"elapsed time: {toc-tic} s")

# 3. Visualize the results

In [22]:
# Load the JSON file
with open("output.json", "r") as file:
    data = json.load(file)

# Extract the trajectory data 
wellbores = data["data"]["Curves"]

# Cost contour data
contour_1site=data['data']['CostASite']

## 3.1 check the data

In [ ]:
# Loop through each wellbore
for wellbore in wellbores:
    # check the target location
    print((wellbore['EAST'][-1], wellbore['NORTH'][-1], wellbore['TVD'][-1]))

input_data['FIELDOPT INPUT BLOCK']['PTM']

(517167.9, 6782513.9, -1550.0)
(518255.3, 6784864.0, -1559.0)
(516179.0, 6782799.0, -1557.0)
(515726.0, 6784079.0, -1577.0)


{'DESCRIPTION': 'target location, i.e., the 1st point of completion interval. 3D, [EAST,NORTH,Depth]',
 'UNIT': 'm',
 'VALUE': [[517167.9, 6782513.9, -1550.0],
  [518255.3, 6784864.0, -1559.0],
  [516179.0, 6782799.0, -1557.0],
  [515726.0, 6784079.0, -1577.0]]}

## 3.2 3D Visualization

In [23]:
# Plot wellbore trajectory
fig1=None #initiate a fig
styles=['k-', 'r-', 'g-', 'b-'] # line style for each trajectory 
for i, wellbore in enumerate(wellbores):
    fig1=PlotSurvey(wellbore, # trajectory data
                    fig=fig1, # figure handle
                    style=styles[i], # line style for the trajectory
                    show=0) # wait for plottiing all trajectories

# plot the cost contour for the site
PlotContour(X=contour_1site['X'],
            Y=contour_1site['Y'],
            Contour_Val=contour_1site['cost'],
            fig=fig1,
            name='cost contour for site',
            azim=-65, # view angle azimuth
            elev=40, # view angle elevation
            show=0)

# show the figure
fig1.show()